In [ ]:
# cd ~/twinews-logs ; jupython --venv st-venv ~/notebooks/twinews/dump.ipynb

In [1]:
isNotebook = '__file__' not in locals()

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *

In [3]:
import sys ; sys.path.append(homeDir() + '/Workspace/Python/Datasets/Twinews')
from twinews.utils import *

In [4]:
logger = Logger(tmpDir('logs') + "/twinews-dump.log") if isNotebook else Logger("dump.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [5]:
(hjuser, hjpass, hjhost) = getMongoAuth(user='hayj')

In [6]:
newsCollection = getNewsCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)
usersCollection = getUsersCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)

twinews news (version 1.0) initialised.
twinews users (version 1.0) initialised.


In [7]:
outputDir = nosaveDir() + "/twinews-dumps/" + getDateSec()

In [8]:
def ndjsonDump(gen, outputDir, maxMoSize=20, refreshEach=10, size=None,
               printRatio=0.01, logger=None, verbose=True):
    mkdir(outputDir)
    i = -1
    currentFile = None
    pbar = None
    if size is None:
        try:
            size = len(gen)
        except: pass
    if size is not None:
        pbar = ProgressBar(size, logger=logger, verbose=verbose, printRatio=printRatio)
    for row in gen:
        if currentFile is None or currentFile.getEstimatedMoSize(refreshEach=refreshEach) > maxMoSize:
            if currentFile is not None:
                currentFile.close()
            i += 1
            currentPath = outputDir + "/" + str(i) + ".ndjson.bz2"
            assert not isFile(currentPath)
            currentFile = NDJson(currentPath, closeAtEachAppend=False)
        currentFile.append(row)
        if pbar is not None:
            pbar.tic()
    if currentFile is not None:
        currentFile.close()

In [12]:
def gen(collection):
    for row in collection.find():
        del row['_id']
        yield row

In [10]:
ndjsonDump(gen(newsCollection), outputDir + "/news", size=len(newsCollection), refreshEach=100, logger=logger)

  0% [                    ]
 10% [==                  ] (21m 0.72s left)
 20% [====                ] (20m 16.719s left)
 30% [======              ] (18m 42.73s left)
 40% [========            ] (15m 56.414s left)
 50% [==========          ] (13m 26.019s left)
 60% [============        ] (10m 41.233s left)
 70% [==============      ] (7m 58.255s left)
 80% [================    ] (5m 15.997s left)
 90% [==================  ] (2m 37.022s left)
100% [====================] (total duration: 26m 41.66s, mean duration: 0.002s)


In [13]:
ndjsonDump(gen(usersCollection), outputDir + "/users", size=len(usersCollection), refreshEach=3, logger=logger)

  0% [                    ]
  9% [=                   ] (20m 32.358s left)
 19% [===                 ] (18m 20.929s left)
 29% [=====               ] (16m 8.97s left)
 39% [=======             ] (13m 37.322s left)
 49% [=========           ] (11m 10.865s left)
 59% [===========         ] (8m 58.919s left)
 69% [=============       ] (6m 44.934s left)
 79% [===============     ] (4m 30.238s left)
 89% [=================   ] (2m 14.563s left)
 99% [=================== ] (0.164s left)
100% [====================] (total duration: 22m 15.509s, mean duration: 0.041s)
